# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
# run end to end
# fix hdbscan with existing db somehow


In [2]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [3]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [4]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [5]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

08:45:37 | NewsletterAgent.newsletter_agent | INFO | Test info message
08:45:37 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
08:45:37 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [6]:
print("🚀 Creating NewsletterAgent...")

do_download=False
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251008074920840197'
    step_name = 'step_04_extract_summaries'
#     del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

08:45:48 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Using provided state with 166 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251008074920840197
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251008074920840197


In [7]:
agent.state.get_status()


{'headlines': {'total': 166},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_05_rate_articles',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 44.44444444444444,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [10]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


07:49:39 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting check_workflow_status
07:49:39 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Completed check_workflow_status


⏱️  Total execution time: 6.65s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- Status: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step details:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Would you like me to start Step 1 (gather URLs) and continue through the workflow, or run a specific step?


In [11]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


07:49:46 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting check_workflow_status
07:49:46 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Completed check_workflow_status
07:49:47 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting Step 1: Gather URLs
07:49:47 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Cleaning download/sources: 
07:49:47 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Successfully cleaned download/sources
07:49:47 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Cleaning download/text: 
07:49:47 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Successfully cleaned download/text
07:49:47 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Cleaning download/html: 
07:49:47 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Successfully cleaned download/html
2025-10-08 07:49:47,540 - fetcher_5754795408 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-08 07:49:47,710 - fetcher_5754795408 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-08 07:49:47,711 - fetcher_5754795408 - INFO - [fetch_rss] Fetching RSS from Techmeme: https://www.techmeme.com/feed.xml
2025-10-08 07:49:47,850 - fetcher_5754795408 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-08 07:49:47,852 - fetcher_5754795408 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-08 07:49:47,942 - fetcher_5754795408 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2025-10-08 07:49:47,960 - fetcher_5754795408 - INFO - [fetch_rss] RSS fetch successful for The Register: 50 articles
2025-10-08 07:49:47,960 - fetcher_5754795408 - INFO - [fetch_html] Fetching HTML from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-10-08 07:49:47,960 - fetcher_5754795408 - INFO - [fetch_html] Fetching HTML from VentureBeat: htt

2025-10-08 07:49:49,476 - fetcher_5754795408 - INFO - [fetch_html] Source dict for Bloomberg: {'type': 'html', 'url': 'https://www.bloomberg.com/ai', 'filename': 'Bloomberg', 'include': ['^https://www.bloomberg.com/news/'], 'rss': 'https://feeds.bloomberg.com/technology/news.rss'}
2025-10-08 07:49:49,476 - fetcher_5754795408 - INFO - Starting scrape_source https://www.bloomberg.com/ai, Bloomberg
2025-10-08 07:49:49,477 - fetcher_5754795408 - INFO - scrape_url(https://www.bloomberg.com/ai)
2025-10-08 07:49:49,477 - fetcher_5754795408 - INFO - scraping https://www.bloomberg.com/ai to download/sources
2025-10-08 07:49:49,477 - fetcher_5754795408 - INFO - Downloading https://www.bloomberg.com/ai
2025-10-08 07:49:49,479 - fetcher_5754795408 - INFO - [fetch_html] Source dict for Business Insider: {'type': 'html', 'url': 'https://www.businessinsider.com/tech', 'filename': 'Business_Insider', 'exclude': ['^https://www.insider.com', '^https://www.passionfroot.me']}
2025-10-08 07:49:49,479 - fet

...

2025-10-08 07:50:20,291 - fetcher_5754795408 - DEBUG - Added UTC timezone: 2025-10-08 07:50:20+00:00
2025-10-08 07:50:20,291 - fetcher_5754795408 - DEBUG - Converted to UTC: 2025-10-08 07:50:20+00:00
2025-10-08 07:50:20,291 - fetcher_5754795408 - DEBUG - Formatted last_updated: 2025-10-08T07:50:20Z
2025-10-08 07:50:20,291 - fetcher_5754795408 - INFO - Saving HTML to download/sources/Reddit.html
2025-10-08 07:50:20,295 - fetcher_5754795408 - INFO - [fetch_html] Parsing HTML file: download/sources/Reddit.html
2025-10-08 07:50:20,320 - fetcher_5754795408 - INFO - [fetch_html] Parsed HTML file: download/sources/Reddit.html
2025-10-08 07:50:20,320 - fetcher_5754795408 - INFO - [fetch_html] HTML fetch successful for Reddit: 60 articles
2025-10-08 07:50:21,029 - fetcher_5754795408 - DEBUG - Found last updated time from HTTP header: Tue, 07 Oct 2025 15:57:54 -0700
2025-10-08 07:50:21,029 - fetcher_5754795408 - DEBUG - Attempting to parse last_updated: 'Tue, 07 Oct 2025 15:57:54 -0700' (type: <

,source,url
0,Ars Technica,20
1,Bloomberg,27
2,Business Insider,17
3,FT,44
4,Feedly AI,73
5,Hacker News,29
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,100
9,Reddit,49


,source,title,url,published,rss_summary,id
0,Ars Technica,Ars Live: Is the AI bubble about to pop? Ed Zi...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Tue, 07 Oct 2025 19:15:26 +0000","Our discussion happens today, October 7, at 3:...",0
1,Ars Technica,Dead celebrities are apparently fair game for ...,https://arstechnica.com/ai/2025/10/dj-bruce-le...,"Tue, 07 Oct 2025 17:08:05 +0000","OpenAI's likeness protections for living ""publ...",1
2,Ars Technica,OpenAI wants to make ChatGPT into a universal ...,https://arstechnica.com/ai/2025/10/openai-want...,"Mon, 06 Oct 2025 19:10:44 +0000","Spotify, Canva, Zillow among today's launch pa...",2
3,Ars Technica,Deloitte will refund Australian government for...,https://arstechnica.com/ai/2025/10/deloitte-wi...,"Mon, 06 Oct 2025 16:30:29 +0000",Consulting firm quietly admitted to GPT-4o use...,3
4,Ars Technica,AMD wins massive AI chip deal from OpenAI with...,https://arstechnica.com/ai/2025/10/amd-wins-ma...,"Mon, 06 Oct 2025 14:45:28 +0000",ChatGPT maker will be allowed to buy 10% of AM...,4
...,...,...,...,...,...,...
604,NewsAPI,Why the Future of Automation Is Teaching Machi...,http://www.pymnts.com/artificial-intelligence-...,2025-10-07T08:00:02Z,NaN,604
605,NewsAPI,Over half of India’s higher education institut...,https://www.thehindubusinessline.com/news/educ...,2025-10-07T10:07:03Z,NaN,605
606,NewsAPI,"Nagarro plans to add 1,000 professionals in In...",https://www.thehindubusinessline.com/info-tech...,2025-10-07T10:02:02Z,NaN,606
607,NewsAPI,IndiaAI scales up efforts for deepfake detecti...,https://economictimes.indiatimes.com/tech/arti...,2025-10-07T08:48:33Z,NaN,607


07:50:21 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Completed Step 1: Gathered 696 articles
07:50:22 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting check_workflow_status
07:50:22 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Completed check_workflow_status


⏱️  Total execution time: 40.60s
📊 Final result:
Step 1 completed successfully.

Summary:
- Fetched articles: 696 headlines from 17 RSS sources
- Articles stored in persistent state: 609
- Workflow progress: 1/9 steps complete (11.1%)
- Next step: Step 2 — Filter URLs (AI-related only)

Would you like me to proceed and run Step 2 now?


In [12]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,Ars Live: Is the AI bubble about to pop? Ed Zi...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Tue, 07 Oct 2025 19:15:26 +0000","Our discussion happens today, October 7, at 3:...",0
1,Ars Technica,Dead celebrities are apparently fair game for ...,https://arstechnica.com/ai/2025/10/dj-bruce-le...,"Tue, 07 Oct 2025 17:08:05 +0000","OpenAI's likeness protections for living ""publ...",1
2,Ars Technica,OpenAI wants to make ChatGPT into a universal ...,https://arstechnica.com/ai/2025/10/openai-want...,"Mon, 06 Oct 2025 19:10:44 +0000","Spotify, Canva, Zillow among today's launch pa...",2
3,Ars Technica,Deloitte will refund Australian government for...,https://arstechnica.com/ai/2025/10/deloitte-wi...,"Mon, 06 Oct 2025 16:30:29 +0000",Consulting firm quietly admitted to GPT-4o use...,3
4,Ars Technica,AMD wins massive AI chip deal from OpenAI with...,https://arstechnica.com/ai/2025/10/amd-wins-ma...,"Mon, 06 Oct 2025 14:45:28 +0000",ChatGPT maker will be allowed to buy 10% of AM...,4
...,...,...,...,...,...,...
604,NewsAPI,Why the Future of Automation Is Teaching Machi...,http://www.pymnts.com/artificial-intelligence-...,2025-10-07T08:00:02Z,NaN,604
605,NewsAPI,Over half of India’s higher education institut...,https://www.thehindubusinessline.com/news/educ...,2025-10-07T10:07:03Z,NaN,605
606,NewsAPI,"Nagarro plans to add 1,000 professionals in In...",https://www.thehindubusinessline.com/info-tech...,2025-10-07T10:02:02Z,NaN,606
607,NewsAPI,IndiaAI scales up efforts for deepfake detecti...,https://economictimes.indiatimes.com/tech/arti...,2025-10-07T08:48:33Z,NaN,607


In [14]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
8,NewsAPI,100
4,Feedly AI,73
6,HackerNoon,50
12,The Register,50
9,Reddit,49
3,FT,44
16,Washington Post,29
5,Hacker News,29
13,The Verge,28
1,Bloomberg,27


In [15]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:51:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting Step 2: Filter URLs
07:51:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 🔍 Filtering 609 headlines...
07:51:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
07:51:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 🔍 Filtering 609 articles for dupes.
07:51:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | URL deduplication: 271 duplicates filtered, 338 new URLs remain
07:51:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 🔍 Filtering 609 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/ai/2025/10/ars-live-is-the-ai-bubble-about-to-pop-a-live-chat-with-ed-zitron/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/dj-bruce-lee-and-jackass-mr-rogers-dead-celebrities-become-puppets-in-sora-2-videos/
url not found - inserting new URL
checking https://arstechnica.com/ai/2025/10/openai-wants-to-make-chatgpt-into-a-universal-app-frontend/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/deloitte-will-refund-australian-government-for-ai-hallucination-filled-report/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/amd-wins-massive-ai-chip-deal-from-openai-with-stock-sweetener/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/openai-jony-ive-struggle-with-technical-details-on-secretive-new-ai-gadget/
found url, no cutoff set - treating 

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
07:51:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Processing 14 chunks with concurrency 16
07:51:21 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Completed Step 2: 168 AI-related articles, 271 duplicates removed


⏱️  Total execution time: 18.15s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 271 duplicate URLs, classified 338 new articles, found 168 AI-related.


In [16]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

07:51:30 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting Step 3: Download Articles
07:51:30 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting concurrent scraping of 168 AI-related articles


▶ Starting Step 3: step_03_download_articles


07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Launching browser for 168 URLs with 16 concurrent workers
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 0 fetching 1 of 168 https://www.bloomberg.com/news/articles/2025-10-07/data-center-boom-brings-risks-of-overbuilding-ares-deveer-says
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Skipping ignored domain: www.bloomberg.com
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 0 completed https://www.bloomberg.com/news/articles/2025-10-07/data-center-boom-brings-risks-of-overbuilding-ares-deveer-says with status: skipped
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 0 fetching 2 of 168 https://poprant.indiatimes.com/trending/taylor-swift-loses-over-1000000-instagram-followers-after-the-life-of-a-showgirl-release-as-fans-slam-use-of-ai-in-album-promotion-672679.html
07:51:31 | NewsletterAgent.te

07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 8 fetching 12 of 168 http://www.techmeme.com/251008/p20#a251008p20
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | scrape_url(http://www.techmeme.com/251008/p20#a251008p20)
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | scraping http://www.techmeme.com/251008/p20#a251008p20 to download/html
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Downloading http://www.techmeme.com/251008/p20#a251008p20
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 9 fetching 13 of 168 https://fortune.com/2025/10/07/honeywell-international-artificial-intelligence-manufacturing-operations/
07:51:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | scrape_url(https://fortune.com/2025/10/07/honeywell-international-artificial-intelligence-manufacturing-operations/)
07:51:31 | NewsletterAgent.test_newsletter_2025100

07:51:56 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Downloading https://hackernoon.com/can-a-machine-learning-algorithm-predict-your-income-just-from-your-demographics?source=rss
07:51:57 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Response: 200
07:51:59 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Saving HTML to download/html/Coaio_Sounds_Alarm__Massive_AI_Shifts_and_New_Tool_Rules_Weaponising_Software_Development.html
07:51:59 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 3 completed https://www.ibtimes.com/coaio-sounds-alarm-massive-ai-shifts-new-tool-rules-weaponising-software-development-3785915 with status: 200
07:51:59 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 3 fetching 21 of 168 http://www.techmeme.com/251008/p22#a251008p22
07:51:59 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | scrape_url(http://www.techmeme.com/251008/p22#a251008p22)
07:51:59 | N

...

08:02:51 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Saving HTML to download/html/IndiaAI_scales_up_efforts_for_deepfake_detection__bias_mitigation__AI_penetration_testing.html
08:02:53 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 2 completed https://economictimes.indiatimes.com/tech/artificial-intelligence/indiaai-scales-up-efforts-for-deepfake-detection-bias-mitigation-ai-penetration-testing/articleshow/124358160.cms with status: 200
08:02:53 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 2 fetching 165 of 168 https://www.infoworld.com/article/4069470/with-arduino-deal-qualcomm-pushes-deeper-into-open-source-and-edge-ai-development.html
08:02:53 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | scrape_url(https://www.infoworld.com/article/4069470/with-arduino-deal-qualcomm-pushes-deeper-into-open-source-and-edge-ai-development.html)
08:02:53 | NewsletterAgent.test_newsletter_20251008074920840197 | INF

08:04:08 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Page URL redirected from http://www.techmeme.com/251008/p10#a251008p10 to https://www.techmeme.com/251008/p10#a251008p10
08:04:09 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Saving HTML to download/html/An_overview_of_detailed_AI_usage_reports_from_OpenAI_and_others__as_Microsoft_s_AI_for_Good_Lab_estimates_that_15__of_the_world_s_working_population_is_using_AI__Financial_Times.html
08:04:09 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 9 completed http://www.techmeme.com/251008/p10#a251008p10 with status: 200
08:04:12 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Saving HTML to download/html/NVDA__Nvidia_Deals_Ignite_Iris_Energy_Stock_Rally_With_Double-Digit_Surge.html
08:04:13 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Worker 14 completed https://finance.yahoo.com/news/nvda-nvidia-deals-ignite-iris-143245180.html with status: 200

Starting with 168 rows...
Processing 168 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

08:04:57 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Completed Step 3: Downloaded 144 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 34 vs 95
    34: Feedly AI - Q&A with Sam Altman on OpenAI's unifying vision, infrastructure deals, the investor mindset, ChatGPT apps, Instant Checkout, Sora, copyright, feedback, and more (Ben Thompson/Stratechery)
    95: Techmeme - Q&A with Sam Altman on OpenAI's unifying vision, infrastructure deals, the investor mindset, ChatGPT apps, Instant Checkout, Sora, copyright, feedback, and more (Ben Thompson/Stratechery)
  Pair: 60 vs 96
    60: Feedly AI - Researchers are investigating using synthetic diamonds to help cool AI data center chips; De Beers-owned Element Six has long used diamonds in satellite chips (Amos Zeeberg/New York Times)
    96: Techmeme - Researchers are investigating using synthetic diamonds to help cool AI data center chips; De Beers-owned Element Six has long used diamonds in satellite chips (Amos Zeeberg/New York Times)
Removing 2 rows due to high simi

In [17]:
headline_df = state.headline_df
headline_df.loc[headline_df["status"]!=200]



,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation
142,Bloomberg,"At $1.2 Trillion, More High-Grade Debt Now Tie...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,43,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
143,WSJ,European Union Outlines Strategies to Boost AI...,https://www.wsj.com/tech/ai/european-union-out...,NaN,NaN,442,True,skipped,https://www.wsj.com/tech/ai/european-union-out...,,None,,0,wsj.com,The Wall Street Journal,5.0
144,Bloomberg,Soaring AI Valuations Spur Market Correction R...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,21,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
145,Bloomberg,AI Power Boom Turns Nuclear Stocks Into the Ma...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,23,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
146,Bloomberg,AI Fintech Optasia Targets Biggest South Afric...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,25,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
147,Bloomberg,Anthropic Opening Its First India Office to Ta...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,26,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
148,Bloomberg,xAI to Raise $20 Billion After Nvidia and Othe...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,27,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
149,Bloomberg,"CareerBuilder, Job Search Site Toppled by AI, ...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,28,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
150,Bloomberg,"The Big TakeOpenAI, Nvidia Fuel $1 Trillion AI...",https://www.bloomberg.com/news/features/2025-1...,NaN,NaN,29,True,skipped,https://www.bloomberg.com/news/features/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
151,Bloomberg,Dell Boosts Forecast Through 2030 Off AI Boom ...,https://www.bloomberg.com/news/videos/2025-10-...,NaN,NaN,34,True,skipped,https://www.bloomberg.com/news/videos/2025-10-...,,None,,0,bloomberg.com,Bloomberg,5.0


In [18]:
headline_df = state.headline_df
headline_df.loc[headline_df["html_path"]==""]



,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation
142,Bloomberg,"At $1.2 Trillion, More High-Grade Debt Now Tie...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,43,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
143,WSJ,European Union Outlines Strategies to Boost AI...,https://www.wsj.com/tech/ai/european-union-out...,NaN,NaN,442,True,skipped,https://www.wsj.com/tech/ai/european-union-out...,,None,,0,wsj.com,The Wall Street Journal,5.0
144,Bloomberg,Soaring AI Valuations Spur Market Correction R...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,21,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
145,Bloomberg,AI Power Boom Turns Nuclear Stocks Into the Ma...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,23,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
146,Bloomberg,AI Fintech Optasia Targets Biggest South Afric...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,25,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
147,Bloomberg,Anthropic Opening Its First India Office to Ta...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,26,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
148,Bloomberg,xAI to Raise $20 Billion After Nvidia and Othe...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,27,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
149,Bloomberg,"CareerBuilder, Job Search Site Toppled by AI, ...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,28,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
150,Bloomberg,"The Big TakeOpenAI, Nvidia Fuel $1 Trillion AI...",https://www.bloomberg.com/news/features/2025-1...,NaN,NaN,29,True,skipped,https://www.bloomberg.com/news/features/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
151,Bloomberg,Dell Boosts Forecast Through 2030 Off AI Boom ...,https://www.bloomberg.com/news/videos/2025-10-...,NaN,NaN,34,True,skipped,https://www.bloomberg.com/news/videos/2025-10-...,,None,,0,bloomberg.com,Bloomberg,5.0


In [8]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:27:13 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Processing 166 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
08:27:13 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Using model 'gpt-4.1-mini' for summarization
08:27:13 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Initialized LLMagent:
system_prompt: You are an expert AI news analyst. Your task is to create concise, informative bullet-point summaries of AI and technology articles for a professional newsletter audience.

You will receive a list of JSON object with fields "id" and "title"
Return **only** a JSON object that satisfies the provided schema.
For each article provided, you MUST return one element with the same id, and the summary.
Return elements in the same order t

▶ Starting Step 4: step_04_extract_summaries


08:27:14 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Summarize the article below:

[
  {
    "id":212,
    "text_content":"Page title: The Dark Side of GPT: How NLP Tools Are Being Used for Phishing, Scams, and Misinformation | HackerNoon\nSocial card title: The Dark Side of GPT: How NLP Tools Are Being Used for Phishing, Scams, and Misinformation | HackerNoon\nSocial card description: Explore how GPT and NLP tools are fueling phishing, scams, and misinformation. Learn how AI-powered attacks are becoming more sophisticated and dangerous.\nPhishing has long been a go-to tactic for cybercriminals looking to steal sensitive information, from login credentials to personal financial data. Traditionally, phishing attempts were relatively easy to spot, often riddled with suspicious grammar or generic phrases like \"Dear Sir\/Madam\" and \"Click here to claim your prize!\" However, with the advent of advanced language models like GPT-3 and GPT-4, phishing atta

08:27:14 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Summarize the article below:

[
  {
    "id":132,
    "text_content":"Page title: JP Morgan CEO Jamie Dimon says bank\u2019s AI investment has already paid off | Semafor\nSocial card title: JP Morgan CEO says bank\u2019s AI investment has already paid off\nSocial card description: The company invests about $2 billion a year in AI, but has reduced headcount and improved efficiency to save at least the same amount, Dimon said.\nJPMorgan CEO Jamie Dimon said the money the bank has poured into AI has already begun paying for itself.\nThe company invests about $2 billion a year in AI, but has reduced headcount and improved efficiency to save at least the same amount, Dimon told Bloomberg.\nThe CEO has long been bullish on AI, suggesting it could mean that future generations \u201care going to live to 100 [and]... probably be working three-and-a-half days a week.\u201d\nStill, \u201cbusinesses aren\u2019t 

08:27:14 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Summarize the article below:

[
  {
    "id":214,
    "text_content":"Page title: I Built a 'God-Mode' AI Prompt to Master SEO and AI Search\u2014And I'm Giving It Away | HackerNoon\nSocial card title: I Built a 'God-Mode' AI Prompt to Master SEO and AI Search\u2014And I'm Giving It Away | HackerNoon\nSocial card description: Stop choosing between SEO and optimizing for AI search. This comprehensive prompt turns ChatGPT into a content optimization expert for both worlds.\nLet's be honest: keeping up with content optimization is exhausting. For years, we\u2019ve been chasing Google's algorithm, tweaking keywords, and building backlinks. Just when we thought we had a handle on it, the entire game changed. Now, we\u2019re not just writing for Google; we\u2019re writing for ChatGPT, Perplexity, AI Overviews, and a dozen other AI-powered answer engines.\nAs a builder and marketer, I found myself stuck. My

...

08:29:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DistilledStory(id=568, short_summary='Reuben Rothstein encountered AI in his 2019 legal work, illustrating how AI both challenges and aids the next generation by altering job roles, skills requirements and workplace dynamics, and prompting updated workforce training and legal practice adaptation.')]
08:29:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Read the news item summary below, and output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text.
[
  {
    "id":140,
    "input_text":"Grasshopper streamlines credit memo writing with AI\n\nGrasshopper Bank\u2019s commercial lending clients want streamlined decisioning, so the bank is looking to AI to speed up the lending process\n\n- Grasshopper Bank is implementing AI to streamline credit memo writing and accelerate the lending decision process for its commercial client

08:29:03 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Read the news item summary below, and output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text.
[
  {
    "id":25,
    "input_text":"AI Fintech Optasia Targets Biggest South Africa IPO in 2025\n\n- no content"
  }
]
08:29:04 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DistilledStory(id=570, short_summary='Mastercard says it is integrating artificial intelligence into business payments to improve liquidity management, speed and execution of transactions, and cost efficiency, signaling a strategic shift to maintain competitiveness in corporate finance.')]
08:29:04 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Read the news item summary below, and output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text.
[
  {
    "id":26,
    "input_text":"Anthrop

08:29:09 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Read the news item summary below, and output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text.
[
  {
    "id":445,
    "input_text":"Dell Lifts Long-Term Growth Targets on Surging AI Demand\n\n- no content"
  }
]
08:29:09 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DistilledStory(id=557, short_summary='OpenAI said on Tuesday it identified and halted misuse of its ChatGPT model by an actor likely linked to the Chinese government that sought help drafting a surveillance proposal, highlighting risks of state-linked exploitation.')]
08:29:09 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Read the news item summary below, and output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text.
[
  {
    "id":447,
    "input_text":"OpenAI, AMD Announce Massive C

08:29:19 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DistilledStory(id=108, short_summary='Penguin Solutions shares fell more than 20%, sharply reducing market value for the AI infrastructure maker.')]
08:29:21 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DistilledStory(id=143, short_summary='Spend-management platform Ramp Inc. has launched AI agents to detect and prevent invoice fraud for corporate customers, strengthening its bill-payment security amid rising invoice scams enabled by generative AI.')]


✅ Completed Step 4: Generated AI summaries for 166/166 articles
⏱️  Total execution time: 128.16s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 166/166 articles.
💾 Summaries stored in headline DataFrame.


In [ ]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



In [ ]:
# remove
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [10]:
state.headline_df["short_summary"] 


0      A secondary analysis of the FDP-PONV trial (n=...
1      Advanced GPT models like GPT-3 and GPT-4 are b...
2      JPMorgan CEO Jamie Dimon says the bank's rough...
3      A developer built a comprehensive AI prompt th...
4      Wells Fargo analyst Ryan MacWilliams says agen...
                             ...                        
161    Recruiters increasingly use AI to scan résumés...
162    Ares says the current data-center construction...
163    AI-driven power demand is expected to prompt a...
164    BlackRock's new investment bets position the f...
165    Penguin Solutions shares fell more than 20%, s...
Name: short_summary, Length: 166, dtype: object

In [8]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:46:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Rating 166 AI articles using fn_rate_articles
08:46:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Calculating article ratings for 166 articles
08:46:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Rating recency
08:46:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Rating spam probability
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles


08:46:07 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | low quality articles: {0.0: 125, 1.0: 18, 0.9890129880157539: 2, 0.9859363206911979: 1, 0.9999957534718431: 1, 0.999999448776502: 1, 0.9999937270200764: 1, 0.9999998063873693: 1, 0.999980972389532: 1, 0.9999909853565219: 1, 0.9999897933314713: 1, 7.05287985111216e-11: 1, 1.2917800047859961e-12: 1, 1.9555681087850496e-08: 1, 0.49999999904767284: 1, 7.287724095819692e-14: 1, 2.3659776091347615e-14: 1, 6.914400106940203e-13: 1, 5.3318128479527204e-14: 1, 0.9525741086376607: 1, 4.691164021834418e-08: 1, 0.5926665998515953: 1, 0.9399132588278407: 1}
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240
08:46:15 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | on topic articles: {1.0: 117, 0.9999998063873693: 3, 0.7310585784564795: 3, 0.999876564755060

08:47:25 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | total battles: 396
08:47:25 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Recomputing Bradley-Terry ratings
08:47:25 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Recomputed Bradley-Terry ratings
08:47:25 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Computed Bradley-Terry z-scores


Battle: 5 beat 2
Battle: 5 beat 1
Battle: 5 beat 0
Battle: 5 beat 4
Battle: 5 beat 3
Battle: 2 beat 1
Battle: 2 beat 0
Battle: 2 beat 4
Battle: 2 beat 3
Battle: 1 beat 0
Battle: 1 beat 4
Battle: 1 beat 3
Battle: 0 beat 4
Battle: 0 beat 3
Battle: 4 beat 3
Battle: 6 beat 9
Battle: 6 beat 8
Battle: 6 beat 10
Battle: 6 beat 7
Battle: 6 beat 11
Battle: 9 beat 8
Battle: 9 beat 10
Battle: 9 beat 7
Battle: 9 beat 11
Battle: 8 beat 10
Battle: 8 beat 7
Battle: 8 beat 11
Battle: 10 beat 7
Battle: 10 beat 11
Battle: 7 beat 11
Battle: 13 beat 15
Battle: 13 beat 14
Battle: 13 beat 16
Battle: 13 beat 17
Battle: 13 beat 12
Battle: 15 beat 14
Battle: 15 beat 16
Battle: 15 beat 17
Battle: 15 beat 12
Battle: 14 beat 16
Battle: 14 beat 17
Battle: 14 beat 12
Battle: 16 beat 17
Battle: 16 beat 12
Battle: 17 beat 12
Battle: 20 beat 21
Battle: 20 beat 23
Battle: 20 beat 18
Battle: 20 beat 19
Battle: 20 beat 22
Battle: 21 beat 23
Battle: 21 beat 18
Battle: 21 beat 19
Battle: 21 beat 22
Battle: 23 beat 18
Battl

...

Battle: 69 beat 30
Battle: 69 beat 113
Battle: 69 beat 101
Battle: 69 beat 98
Battle: 69 beat 44
Battle: 30 beat 113
Battle: 30 beat 101
Battle: 30 beat 98
Battle: 30 beat 44
Battle: 113 beat 101
Battle: 113 beat 98
Battle: 113 beat 44
Battle: 101 beat 98
Battle: 101 beat 44
Battle: 98 beat 44
Battle: 60 beat 142
Battle: 60 beat 61
Battle: 60 beat 126
Battle: 60 beat 5
Battle: 60 beat 35
Battle: 142 beat 61
Battle: 142 beat 126
Battle: 142 beat 5
Battle: 142 beat 35
Battle: 61 beat 126
Battle: 61 beat 5
Battle: 61 beat 35
Battle: 126 beat 5
Battle: 126 beat 35
Battle: 5 beat 35
Battle: 20 beat 131
Battle: 20 beat 53
Battle: 20 beat 76
Battle: 20 beat 143
Battle: 20 beat 92
Battle: 131 beat 53
Battle: 131 beat 76
Battle: 131 beat 143
Battle: 131 beat 92
Battle: 53 beat 76
Battle: 53 beat 143
Battle: 53 beat 92
Battle: 76 beat 143
Battle: 76 beat 92
Battle: 143 beat 92
Battle: 83 beat 49
Battle: 83 beat 120
Battle: 83 beat 23
Battle: 83 beat 127
Battle: 83 beat 136
Battle: 49 beat 120
Ba

09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Recomputed Bradley-Terry ratings
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Computed Bradley-Terry z-scores


,id,title,bradley_terry,bt_z
69,69,OpenAI Dev Day 2025: ChatGPT becomes the new a...,11.714913,2.550264
30,30,Here's How Authors Included in Anthropic's $1....,9.590143,2.087715
113,113,Introducing the Gemini 2.5 Computer Use model,9.308049,2.026305
60,60,Broadcom's Big AI Move In Japan Could Slash Da...,9.029834,1.965739
142,142,"Amazon, Meta, Microsoft, and Google are gambli...",8.081415,1.759274
...,...,...,...,...
154,154,Baldur's Gate 3 publishing lead sees Elon Musk...,-8.141105,-1.772268
70,70,Opinion | AI isn’t smart enough to destroy us,-8.165953,-1.777678
161,161,"As AI gets more life-like, a new Luddite movem...",-8.557778,-1.862976
162,162,Gold or AI—Which Rally Do You Trust?,-11.745573,-2.556939


09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Top 10 ids: [69, 30, 113, 60, 142, 61, 20, 53, 131, 83]
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | After round 22/33: 
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Number of ranking changes: 76
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Sum of absolute ranking changes: 5302.0 (avg rank chg 32.13)
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | last_two: 30.951515151515153, prev_two: 29.151515151515152
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Increase in avg rank change, stopping
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Low rated articles: 2
09:08:33 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | low rated article: As AI gets more life-like, a new Luddite movement is taking root -0.5442904053990849
09:08:33 | NewsletterAgent

✅ Completed Step 5: Rated 163 articles
⏱️  Total execution time: 1352.35s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 163 articles with average rating 6.2/10.
⭐ High quality articles (≥7.0): 60
💾 Ratings stored in persistent state.


In [9]:
headline_df=state.headline_df
headline_df.sort_values("rating", ascending=False)


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,input_text,age,recency_score,low_quality,on_topic,important,bt_z,adjusted_len,rating,bradley_terry
0,FT,BoE flags risk of ‘sudden correction’ in tech ...,https://www.ft.com/content/fe474cff-564c-41d2-...,None,None,0,True,200,https://www.ft.com/content/fe474cff-564c-41d2-...,download/html/BoE_flags_risk_of__sudden_correc...,...,BoE flags risk of ‘sudden correction’ in tech ...,0.104951,0.859673,0.000000e+00,1.000000,1.000000e+00,4.085788,0.130977,12.076437,5.621924
2,FT,Insurers balk at multibillion-dollar claims fa...,https://www.ft.com/content/0211e603-7da6-45a7-...,None,None,2,True,200,https://www.ft.com/content/0211e603-7da6-45a7-...,download/html/Insurers_balk_at_multibillion-do...,...,Insurers balk at multibillion-dollar claims fa...,0.365171,0.552753,0.000000e+00,1.000000,1.000000e+00,3.936989,0.127429,11.617172,5.166244
1,FT,SoftBank to buy ABB’s robotics unit in $5.4bn ...,https://www.ft.com/content/4080ecae-0654-4597-...,None,None,1,True,200,https://www.ft.com/content/4080ecae-0654-4597-...,download/html/SoftBank_to_buy_ABB_s_robotics_u...,...,SoftBank to buy ABB’s robotics unit in $5.4bn ...,0.210218,0.728814,0.000000e+00,1.000000,1.000000e+00,3.581664,0.119586,11.430063,4.078093
30,Feedly AI,Here's How Authors Included in Anthropic's $1....,https://www.cnet.com/tech/services-and-softwar...,None,None,30,True,200,https://www.cnet.com/tech/services-and-softwar...,download/html/Here_s_How_Authors_Included_in_A...,...,Here's How Authors Included in Anthropic's $1....,0.114778,0.847049,0.000000e+00,1.000000,1.000000e+00,5.381573,0.632052,10.860674,9.590143
20,The Verge,Google’s latest AI model uses a web browser li...,https://www.theverge.com/news/795463/google-co...,None,None,20,True,200,https://www.theverge.com/news/795463/google-co...,download/html/Google_s_latest_AI_model_uses_a_...,...,Google’s latest AI model uses a web browser li...,0.585044,0.333258,0.000000e+00,1.000000,1.000000e+00,4.728323,0.395850,10.457431,7.589627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,NewsAPI,How Structured Data is Key to Leveraging AI in...,https://www.finextra.com/videoarticle/3356/how...,2025-10-07T09:05:08Z,None,164,True,200,https://www.finextra.com/videoarticle/3356/how...,download/html/How_Structured_Data_is_Key_to_Le...,...,How Structured Data is Key to Leveraging AI in...,1.115287,-0.076801,0.000000e+00,0.000004,6.144171e-06,1.762444,0.003029,1.688683,-1.493093
158,Feedly AI,Grasshopper streamlines credit memo writing wi...,https://finainews.com/lending/grasshopper-stre...,None,None,158,True,200,https://finainews.com/lending/grasshopper-stre...,download/html/Grasshopper_streamlines_credit_m...,...,Grasshopper streamlines credit memo writing wi...,1.034650,-0.023732,0.000000e+00,1.000000,1.730986e-14,0.615739,0.000000,1.592007,-5.004768
154,NewsAPI,Baldur's Gate 3 publishing lead sees Elon Musk...,https://www.gamesradar.com/games/baldur-s-gate...,2025-10-07T10:08:13Z,None,154,True,200,https://www.gamesradar.com/games/baldur-s-gate...,download/html/Baldur_s_Gate_3_publishing_lead_...,...,Baldur's Gate 3 publishing lead sees Elon Musk...,1.109581,-0.073143,0.000000e+00,1.000000,1.955568e-08,-0.408403,0.717587,1.236042,-8.141105
155,Reddit,"Taylor Swift loses over 1,000,000 Instagram fo...",https://poprant.indiatimes.com/trending/taylor...,None,None,155,True,200,https://poprant.indiatimes.com/trending/did-ta...,download/html/Taylor_Swift_loses_over_1_000_00...,...,"Taylor Swift loses over 1,000,000 Instagram fo...",1.807255,-0.428532,4.691164e-08,0.999877,9.610239e-05,-0.386918,0.741860,0.926383,-8.075310


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Using model 'gpt-4.1-mini' for topic extraction
09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Processing 163 articles for topic extraction
09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Initialized LLMagent:
system_prompt: 
# Role and Objective
You are an expert AI news analyst. Your task is to extract a list of up to **5** distinct topics from provided news summaries (or an empty list if no topics can be extracted).

# Input Format
You will receive a list of news summary objects in JSON format including fields "id" and "summary".

# Outp

09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":10,
    "input_text":"OpenAI, AMD Announce Massive Computing Deal, Marking New Phase of AI Boom\n- no content"
  },
  {
    "id":11,
    "input_text":"Dell Lifts Estimates for Next Four Years on AI Demand\nno content"
  },
  {
    "id":12,
    "input_text":"CareerBuilder, Job Search Site Toppled by AI, Wraps Liquidation\nno content"
  },
  {
    "id":13,
    "input_text":"Dell Boosts Forecast Through 2030 Off AI Boom | Bloomberg Tech 10\/7\/2025\n- no content"
  },
  {
    "id":14,
    "input_text":"Dell Lifts Long-Term Growth Targets on Surging AI Demand\n- no content"
  },
  {
    "id":15,
    "input_text":"Employees regularly paste company secrets into ChatGPT\n<h4>Microsoft Copilot, not so much<\/h4> <p>Employees could be opening up to OpenAI in ways that put sensitive data at risk. According to a study by security

09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":30,
    "input_text":"Here's How Authors Included in Anthropic's $1.5B AI Piracy Settlement Can File Claims\nReceived your official settlement notice? Here's where to go to file a claim.\n- Anthropic reached a $1.5 billion settlement for illegally downloading nearly 500,000 copyrighted books from Library Genesis and Pirate Library Mirror to train its AI chatbot Claude.\n- Authors with books included on the official list can file claims to receive $3,000 per book; eligibility requires copyright ownership, registration criteria, and an ISBN or ASIN.\n- Claims must be filed by March 23, 2026, through AnthropicCopyrightSettlement.com, marking a significant precedent for AI companies regarding copyright use in training large language models."
  },
  {
    "id":31,
    "input_text":"Virginia's data center construction boom i

09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":40,
    "input_text":"JPMorgan\u2019s Dimon Says AI Cost Savings Now Match Money Spent\n- no content"
  },
  {
    "id":41,
    "input_text":"Alibaba\u2019s Qwen Technology Lead Sets Up In-House Robot AI Team\nno content"
  },
  {
    "id":42,
    "input_text":"An overview of detailed AI usage reports from OpenAI and others, as Microsoft's AI for Good Lab estimates that 15% of the world's working population is using AI (Financial Times)\n<p><a href=\"http:\/\/www.techmeme.com\/251008\/p10#a251008p10\" title=\"Techmeme permalink\"><img height=\"12\" src=\"http:\/\/www.techmeme.com\/img\/pml.png\" style=\"border: none; padding: 0; margin: 0;\" width=\"11\" \/><\/a> <a href=\"https:\/\/www.ft.com\/\">Financial Times<\/a>:<br \/>\n<span style=\"font-size: 1.3em;\"><b><a href=\"https:\/\/t.co\/JNcBxqSLyu\">An overview of de

09:17:10 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":50,
    "input_text":"Brands are losing interest in AI influencers \u2014 but tech gains could bring a new wave of enthusiasm\nSome brands are shying away from working with AI influencers, while others are cautiously optimistic about the tech.\n- Brand partnerships with AI influencers dropped about 30% from Jan-Aug 2025 vs 2024, driven by consumer backlash and underperformance of AI content.\n- Marketing firms report concerns about reputational risk and lack of authenticity with AI creators; some major brands like Dove have banned AI models.\n- Despite hesitation on full AI influencers, 79% of marketers are increasing investment in AI-generated content enhancements, anticipating improved technology making AI content indistinguishable from real."
  },
  {
    "id":51,
    "input_text":"Recruiters Use A.I. to Scan R\u00e

...

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       36
=== Clustering Quality Metrics ===
Number of clusters: 20
Noise points: 90 (55.2%)
Average cluster size: 3.6 ± 1.5
Cluster size range: 2 - 7
=== Quality Scores ===
Silhouette Score: 0.301 (higher is better)
Calinski-Harabasz Score: 7.9 (higher is better)
Davies-Bouldin Score: 1.095 (lower is better)
HDBSCAN Validity Index: 0.147
Composite Score: 0.224 (higher is better)

[I 2025-10-08 09:18:24,298] Trial 198 finished with value: -0.223701030675958 and parameters: {'n_components': 36, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 71 with value: -0.2404575453794539.
=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       266
=== Clustering Quality Metrics ===
Number of clusters: 17
Noise points: 99 (60.7%)
Average cluster size: 3.8 ± 1.8
Cluster size range: 2 - 9
=== Quality Scores ===
Silhouette Score: 0.155 (higher is better)
Calinski-Harabasz Score

09:18:25 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 20: AI Adoption Strategies Across Industries
09:18:25 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | ‘Our chapters will work for any enterprise’: Honeywell’s AI chiefs share the strategies that helped the firm mature its AI efforts (Generative AI, Enterprise AI, AI Frameworks, Business Impact, Honeywell, Gen AI, Governance)
How Visa wove AI into every facet of the company by approaching it as both a science and an art (AI Integration, AI Governance, Financial Services, Generative AI, Cybersecurity, Gen AI, Visa)
How Coca-Cola’s leadership developed a taste for AI that helped distribute the technology across its beverage empire (AI Adoption, Marketing, Supply chain optimization, Beverage Industry, Governance, Coca-Cola, Gen AI)
AI came from tech, but the most advanced AI businesses are in every industry (AI Adoption, Industry Leaders, Gen AI, Healthcare AI, Governance, Tech Giants, Finance)
One com

09:18:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 

09:18:32 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 4: AI Coding Tools and Standards
09:18:32 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | JetBrains backs open AI coding standard that could gnaw at VS Code dominance (Agent Client Protocol, JetBrains, Code Editors, Gen AI, AI Coding Assistants, Open Source, Virtual Assistants)
MCP Servers Show Us the Limits—and the Promise—of LLM-Native Development (MCP Servers, Tool Integration, LLM Development, Developer Tools, Gen AI, Coding Assistants, AI Infrastructure)
What is Model Context Protocol (MCP) and why is it crucial for AI development? (Model Context Protocol, AI Development, Software Supply Chain, Governance, Compliance, Open Source, Cybersecurity)
AI Coding Tools (MCP-series) (Model Context Protocol, Claude, AI Coding Tools, Coding Assistants, Gen AI, Open Source)
09:18:32 | NewsletterAgent.test_newsletter_202510080749208

09:18:41 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 

09:18:42 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 5: Data Centers and Environmental Impact
09:18:42 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Virginia's data center construction boom is even bigger than you think. One company is behind most of it. (Data Centers, Virginia, AI Infrastructure, Amazon, Energy, Big Tech, Infrastructure)
How couldartificial intelligenceimpact air quality? (Data Centers, Power Grid, Pollution Control, Environmental Impact, Energy, Governance, Sustainability)
09:18:42 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 

09:18:43 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | 1: AI Disrupting Tech Industry Jobs
09:18:43 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | CareerBuilder, Job Search Site Toppled by AI, Wraps Liquidation ()
Alibaba’s Qwen Technology Lead Sets Up In-House Robot AI Team ()
0

⏱️  Total execution time: 93.09s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 163 articles into topic clusters.


In [13]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:18:43 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179
09:18:43 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Initialized LLMagent:
system_prompt: # Role & Objective
You are **“The News Pulse Analyst.”**
Your task: read a daily batch of AI-related news items and surface ** 10-30 ** short, high-impact topic titles for an executive summary.
You will receive today's AI-related news items in markdown format.
Each item will have headline, URL, topics, an item rating, and bullet-point summary.
Return ** 10-30 ** distinct, high-impact topics in the supplied JSON format.
Ensure that you propose topics that cover most of the highest-rated items (rated 7 and above)

# Input Format
Headline - Site

Rating: x.x

Topics

▶ Starting Step 7: step_07_select_sections


09:19:35 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: items=['AI Valuation Bubble Risk', 'Insurers Hesitate On AI Liability', 'SoftBank Robotics Acquisition', 'Anthropic Copyright Settlement', 'Gemini 2.5 Computer Use', 'OpenAI ChatGPT App Platform', 'AI-Enabled Phishing Surge', 'Enterprise Data Leakage Via AI', 'Data Center Construction Boom', 'Broadcom NTT Optical Chips', 'AI Infrastructure Energy Strain', 'IBM Agentic AI Expansion', 'Qualcomm Acquires Arduino', 'Agent Client Protocol Adoption', 'EU Apply AI Strategy', 'AI Reliability Engineering', 'Apollo-1 Agent Reliability', 'OpenAI Sora Likeness Loophole', 'OpenAI Hardware Delays', 'Deloitte AI Hallucination Incident', 'OpenAI Agent Builder No-Code', 'Crunch Lab Decentralized AI', 'C2PA Image Provenance', 'Circular Deals Inflate Market']
09:19:35 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Cleaning up initial categories: ['AI Tools and Security Risks', 'AI Adoption Strategies Across 

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_assignment' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_assignment': model=gpt-5-mini, system_len=1945, user_len=176
09:20:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Initialized LLMagent:
system_prompt: You are an AI Topic Router.

Task

Given one news item and a list of candidate topics, output exactly one string: either the single topic whose meaning best matches the news item, or Other if no candidate fits with sufficient confidence.

Rules
	1.	Read fully. Focus on the headline/lede and main subject, not incidental mentions.
	2.	Semantic match. Compare the news item’s meaning to every candidate topic.
	3.	Choose one topic. Pick the topic with the highest semantic overlap with the news item’s main subject.
	4.	Confidence threshold. If your best match has < 60% confidence, output Other.
        - Heuristics:
        - ≥90%: The topic (or a clear synonym) is explici

09:20:31 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: CANDIDATE TOPICS
AI Agents And Reliability
AI Creative Industry Impact
AI Development Tools And Standards
AI Market Valuations
AI Phishing Surge
AI Security Risks
AI Workforce Impact
Anthropic Copyright Settlement
Broadcom NTT Optical Chips
C2PA Image Provenance
Circular Deal Inflation
Cross-Industry AI Adoption
Crunch Lab Decentralized AI
Data Center Environmental Impact
Deepfake Video Ethics
Dell Raises AI Forecasts
Deloitte AI Hallucination
EU AI Strategy
Enterprise AI Data Leakage
Enterprise AI Partnerships
Google AI Web Automation
Google Gemini 2.5
Healthcare AI Investments
Insurers AI Liability
OpenAI Platform Issues
Qualcomm Acquires Arduino
SoftBank Acquires ABB Robotics
Youth Support AI Ethics

Classify the news item into exactly one of the candidate topics above. If your best match is < 60% confidence, output Other.

NEWS ITEM
A.I. Companion Ads for Friend.com Flood NYC Subway, Fueling Back

...

09:21:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Deduplicate the following news articles:
{"id":{"29":29,"114":114,"87":87,"116":116},"extended_summary":{"29":"Here's the pitch deck that Point72-backed Heidi Health used to raise $65 million to battle in the AI scribe race\n\nHeidi Health is gaining traction for ambient clinical documentation against hot startup competitors like Abridge and Ambience Healthcare.\n\nHeidi Health, AI Medical Scribes, Healthcare AI, Funding, Tech, Healthcare, Startups, Clinical Documentation, Virtual Assistants\n\n- Heidi Health raised $65 million in Series B funding led by Point72, valuing the company at $465 million, to advance its AI medical scribe technology amid competition from startups like Abridge and incumbents like Epic.\n- The company differentiates itself by focusing on direct physician adoption and offering a freemium AI scribe model that prioritizes clinician customization rather than deep EHR integration.

09:21:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Deduplicate the following news articles:
{"id":{"20":20,"67":67,"113":113},"extended_summary":{"20":"Google\u2019s latest AI model uses a web browser like you do\n\nGemini\u2019s \u201ccomputer use\u201d model can scroll and click its way through a browser.\n\nGoogle Gemini, Agents, UI Testing, Virtual Assistants, Gen AI, Products, Language Models\n\n- Google introduced Gemini 2.5 Computer Use, an AI model that interacts with web browsers by clicking, scrolling, and typing to perform tasks such as filling out forms and navigating interfaces without API support.\n- This model serves use cases like UI testing and autonomous task execution in browsers, with demonstrated applications in projects like AI Mode and Project Mariner for agentic task completion.\n- Available via Google AI Studio and Vertex AI, Gemini 2.5 supports 13 browser actions and outperforms competitors on web and mobile benchmarks, thou

09:21:00 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | User message: Deduplicate the following news articles:
{"id":{"27":27,"72":72},"extended_summary":{"27":"The iPhone 17 Is Missing a Crucial AI Camera Feature That Google Has Embraced\n\nCommentary: Google is taking the lead on identifying AI image slop by incorporating C2PA photo tagging. Apple can still catch up.\n\nC2PA, Google Pixel, Image Verification, AI Imaging, Disinformation, Ethics, Privacy\n\n- Google Pixel 10 integrates C2PA content credentials to tag images as AI-generated or edited, aiming to combat AI misinformation by embedding provenance metadata in every photo.\n- Apple\u2019s iPhone 17 and iOS 26 lack C2PA implementation despite Apple's significant market share, missing an opportunity to enhance image authenticity verification.\n- Industry experts advocate for broader adoption of C2PA, with Google and Samsung leading, suggesting Apple\u2019s participation could significantly influence market stan

09:21:04 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DupeRecord(id=20, dupe_id=-1), DupeRecord(id=67, dupe_id=20), DupeRecord(id=113, dupe_id=20)]
09:21:04 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DupeRecord(id=21, dupe_id=-1), DupeRecord(id=100, dupe_id=21)]
09:21:05 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DupeRecord(id=95, dupe_id=-1), DupeRecord(id=130, dupe_id=-1)]
09:21:05 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DupeRecord(id=44, dupe_id=-1), DupeRecord(id=92, dupe_id=-1)]
09:21:05 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DupeRecord(id=19, dupe_id=-1), DupeRecord(id=133, dupe_id=19)]
09:21:06 | NewsletterAgent.test_newsletter_20251008074920840197 | INFO | Result: results_list=[DupeRecord(id=6, dupe_id=-1), DupeRecord(id=62, dupe_id=-1)]
09:21:07 | NewsletterAge

✅ Completed Step 7: Categories and article counts:
cat
AI Agents And Reliability              9
AI Creative Industry Impact            9
AI Development Tools And Standards    14
AI Market Valuations                  21
AI Phishing Surge                      2
AI Security Risks                      5
AI Workforce Impact                   13
C2PA Image Provenance                  2
Circular Deal Inflation                1
Cross-Industry AI Adoption            25
Crunch Lab Decentralized AI            1
Data Center Environmental Impact       4
Deepfake Video Ethics                  5
Dell Raises AI Forecasts               1
EU AI Strategy                         2
Enterprise AI Data Leakage             1
Enterprise AI Partnerships             6
Google Gemini 2.5                      1
Healthcare AI Investments              4
OpenAI Platform Issues                 2
Other                                  5
Qualcomm Acquires Arduino              1
SoftBank Acquires ABB Robotics         1
Yo

In [ ]:
state.get_completed_steps() 



In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [14]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"[{row.title}]({row.url}) - {row.site_name}".replace("$","\\\\$")))
#     row_topics = ", ".join(row.topics)
#     display(Markdown(f"Topics: {row_topics}".replace("$","\\\\$")))
    display(Markdown(f"{row.short_summary}".replace("$","\\\\$")))
    display(Markdown(f"{row.input_text}".replace("$","\\\\$")))
    i +=1
#     if i>=30:
#         break

[SoftBank to buy ABB’s robotics unit in \\$5.4bn deal](https://www.ft.com/content/4080ecae-0654-4597-9596-025c373874c3) - Financial Times

SoftBank will acquire ABB's robotics unit for \\$5.4 billion, aligning with founder Masayoshi Son's vision to advance artificial superintelligence and expand SoftBank's robotics and AI capabilities.

SoftBank to buy ABB’s robotics unit in \\$5.4bn deal - Financial Times

Rating: 11.4

Topics: SoftBank, Robotics, Mergers and Acquisitions, Artificial Superintelligence, Hardware, Manufacturing, Japan

Japanese group’s acquisition aligns with founder Masayoshi Son’s vision to realise ‘artificial superintelligence’

- SoftBank is set to acquire ABB's robotics unit for \\$5.4 billion as part of its strategic expansion into robotics technology.
- The acquisition supports SoftBank founder Masayoshi Son's vision of achieving 'artificial superintelligence,' reflecting significant business growth and innovation ambitions.
- This deal positions SoftBank as a major player in the robotics market, with expected implications for advancing AI-integrated robotic solutions across industries.
---


[Google’s latest AI model uses a web browser like you do](https://www.theverge.com/news/795463/google-computer-use-gemini-ai-model-agents) - The Verge

Google introduced Gemini 2.5 Computer Use, an AI model that performs 13 browser actions—clicking, scrolling and typing—to fill forms and navigate interfaces; available in Google AI Studio and Vertex AI, it outperforms competitors on web and mobile benchmarks.

...

Taylor Swift's Instagram followers fell by about 60,000 over 14 days after The Life of a Showgirl's release amid fan backlash over AI-generated promotional content and criticisms of the album's lyrics, despite reports of a million-follower loss.

Taylor Swift loses over 1,000,000 Instagram followers after The Life of a Showgirl release as fans slam use of AI in album promotion - Times of India

Rating: 0.9

Topics: Taylor Swift, AI-generated Content, Social Media Backlash, Music Industry, Pop Rivalry, Artistic Integrity, AI in Music Promotion

Taylor Swift is facing major backlash after releasing The Life of a Showgirl. The major backlash stems from the use of AI in promotional content and criticism over lacklustre lyrics, which has resulted in a significant drop in her Instagram follower count.

- Taylor Swift faced backlash after releasing her album The Life of a Showgirl, primarily due to the use of AI-generated content in promotional materials, notably the "orange door" puzzle and alternate reality game.
- Reports exaggerated a loss of over a million Instagram followers, but precise data shows a drop of about 60,000 followers in 14 days, along with a follower count dip from 282 million to just under that, reflecting rounding effects.
- Fans criticized the AI use as contradictory to Swift's prior advocacy for artistic integrity and raised concerns over the album's perceived lackluster lyrics and an alleged diss track targeting Charli XCX, prompting speculation of a potential public feud.
---


[Penguin Solutions Stock Sinks More Than 20%. What’s Troubling the AI Infrastructure Maker.](https://www.barrons.com/articles/penguin-solutions-stock-ai-outlook-ee9851c6) - Barron's

Penguin Solutions shares fell more than 20%, sharply reducing market value for the AI infrastructure maker.

Penguin Solutions Stock Sinks More Than 20%. What’s Troubling the AI Infrastructure Maker. - Barron's

Rating: 0.7

- no content
---


In [ ]:
class DistilledStory(BaseModel):
    """DistilledStory class for structured output distillation into a single sentence """
    item: str = Field(description="List of StoryRating")
        
system, user, model = LangfuseClient().get_prompt("newsagent/item_distiller")

distill_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=DistilledStory,
            model=model,
            verbose=False,
            logger=logger
        )

In [ ]:
response = await distill_agent.run_prompt(input_text="""AI 'Homeless Man' Challenge Sparks Outrage as Police Called Over Dangerous Viral Trend - International Business Times

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Rating: 1.5

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Parents in shock after dangerous 'homeless man' Snapchat AI prank goes horribly wrong, police called to calm the viral chaos.

A viral AI-driven prank called the 'homeless man' challenge on Snapchat deceived parents into thinking a homeless person had broken into their home, prompting police intervention.
The prank caused significant public backlash due to the misuse of AI technology to create real emergency scares, leading to police questioning the pranksters and debates about the legal consequences.
Experts emphasize the ethical concerns and potential emotional harm caused by such digital pranks, highlighting the need for responsible use of AI to avoid wasting emergency resources and creating community panic.
""")
response

TODO:

- take each summary and boil it down to 1 sentence , output correct format
- initial write sections - prompt and output json for each section asynchronously
- check and rewrite each section for format asynchronsously
- assemble sections
- do a critic loop